In [13]:
# Load processed sorted EC clusters and raw distances

import pickle as pkl
import numpy as np


with open('/home/seyonec/protein-conformal/clean_selection/sorted_dict.pkl', 'rb') as f:
    sorted_dict = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/dists.pkl', 'rb') as f:
    dists = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/true_labels.pkl', 'rb') as f:
    labels = pkl.load(f)

len(sorted_dict), len(dists), len(labels)

(392, 392, 392)

In [14]:
txt = '3.5.4.3'
len(txt.split('.'))

4

In [6]:
from protein_conformal.scope_utils import scope_hierarchical_loss
def get_clean_dict(sorted_ec_dist, dists, labels):
    """

    sorted_ec_dist: a dictionary where each key is a query protein with EC value key,
                    and each value is a dictionary of EC cluster center values and their 
                    euclidean distances to the query protein.

    dists: a 2D numpy array of distances between each test protein embedding and each EC cluster center embedding

    labels: a list of true EC labels, where each element in the list is a list of valid EC labels for the test protein.
            We want to make sure that we check against all the valid EC labels for each test protein, and take the minimum
            loss against the EC cluster centers for each valid EC label.
    """

    num_train_clusters = len(dists[0])
    near_ids = []
    min_sim = np.min(dists)
    max_sim = np.max(dists)

    for i, key in enumerate(sorted_ec_dist):
        #test_id = test_df.loc[true_test_idcs[i], id]
        test_ec = labels[i] # could be list of EC numbers if multiple valid.
        #print(test_ec)
        ec_cluster_centers = [k for k in sorted_ec_dist[key].keys()]
        exact_loss = [[scope_hierarchical_loss(poss_ec, ec_cluster_centers[j]) for poss_ec in test_ec] for j in range(num_train_clusters)]
        # grab the 2nd element in the tuple belonging to each element of exact_loss as mask_exact
        #mask_exact = [x[1] for x in exact_loss]
        loss = [min([y[0] for y in x]) for x in exact_loss]
        mask_exact = [x == 4 for x in loss] ## if loss is 4, then it is an exact match
        
        # define mask_partial as 1 for any element of loss that is <=1 (tolerate retrieving homolog with diff family but same superfamily)
        mask_partial = [l <= 1 for l in loss]
        
        # create a row of size len(lookup_df) where each element is the sum of all entries in S_i until that index
        sum = np.cumsum(dists[i])
        norm_sim = (dists[i] - min_sim) / (max_sim - min_sim) # convert similarities into a probability space (0, 1) based on (min_sim, max_sim)
        #mask_exact = [test_sccs == lookup_df.loc[lookup_idcs[j], 'sccs'] for j in I[i]]

        sum_norm_s_i = np.cumsum(norm_sim)
        near_ids.append({
            'EC_id': key,
            'test_ec': test_ec,
            'EC_centroids': ec_cluster_centers,
            #'meta_query': meta_query,
            'loss' : loss,
            'exact': mask_exact,
            'partial': mask_partial,
            'S_i': dists[i],
            'Sum_i' : sum,
            'Norm_S_i' : norm_sim,
            'Sum_Norm_S_i': sum_norm_s_i,
        })
    return near_ids

In [26]:
clean_dict = get_clean_dict(sorted_dict, dists, labels)

In [27]:
clean_dict[60]['S_i']

array([ 4.35125303,  4.76616192,  5.1906805 , ..., 22.18528366,
       22.23024178, 22.46767235])

In [28]:
np.save('clean_new_v_ec_cluster.npy', clean_dict)


## do the same but for price

In [7]:
# Load processed sorted EC clusters and raw distances

import pickle as pkl
import numpy as np


with open('/home/seyonec/protein-conformal/clean_selection/price_sorted_dict.pkl', 'rb') as f:
    sorted_dict = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/price_dists.pkl', 'rb') as f:
    dists = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/price_true_labels.pkl', 'rb') as f:
    labels = pkl.load(f)

len(sorted_dict), len(dists), len(labels)

(149, 149, 149)

In [8]:
price_clean_dict = get_clean_dict(sorted_dict, dists, labels)

In [9]:
price_clean_dict[60]['S_i']

array([ 3.67167997,  3.79507279,  5.59443903, ..., 19.67150307,
       19.76453018, 19.76453018])

In [6]:
np.save('clean_price_v_ec_cluster.npy', price_clean_dict)


### we run it back but for the ESM embeddings

In [10]:
# Load processed sorted EC clusters and raw distances

import pickle as pkl
import numpy as np


with open('/home/seyonec/protein-conformal/clean_selection/esm_price_sorted_dict.pkl', 'rb') as f:
    esm_sorted_dict_price = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/esm_price_dists.pkl', 'rb') as f:
    esm_dist_price = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/esm_price_true_labels.pkl', 'rb') as f:
    esm_labels_price = pkl.load(f)

len(esm_sorted_dict_price), len(esm_dist_price), len(esm_labels_price)

(149, 149, 149)

In [11]:
esm_price_clean_dict = get_clean_dict(sorted_ec_dist=esm_sorted_dict_price, dists=esm_dist_price, labels=esm_labels_price)

In [14]:
np.save('esm_clean_price_v_ec_cluster.npy', esm_price_clean_dict)


In [15]:
with open('/home/seyonec/protein-conformal/clean_selection/esm_new_sorted_dict.pkl', 'rb') as f:
    esm_sorted_dict_new = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/esm_new_dists.pkl', 'rb') as f:
    esm_dist_new = pkl.load(f)

with open('/home/seyonec/protein-conformal/clean_selection/esm_new_true_labels.pkl', 'rb') as f:
    esm_labels_new = pkl.load(f)

len(esm_sorted_dict_new), len(esm_dist_new), len(esm_labels_new)

(392, 392, 392)

In [16]:
esm_new_clean_dict = get_clean_dict(sorted_ec_dist=esm_sorted_dict_new, dists=esm_dist_new, labels=esm_labels_new)
np.save('esm_clean_new_v_ec_cluster.npy', esm_new_clean_dict)

In [17]:
esm_new_clean_dict[60]['S_i']

array([1.02489674, 1.07202637, 1.61078095, ..., 9.11964321, 9.51962948,
       9.88486385])